In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC,LinearSVC
import numpy as np
import seaborn as sns
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.model_selection import train_test_split

## Support Vector Machines (lineal y polinomico)

In [2]:
#No recuerdo como utilizar el df directamente desde visual studio asi que lo baje a un csv y lo subi de vuelta
df = pd.read_csv("train_data_final.csv")



df.shape

df.columns

Index(['Unnamed: 0', 'customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1',
       'S_3', 'D_41',
       ...
       'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144',
       'D_145', 'target'],
      dtype='object', length=192)

In [3]:
categorical_variables = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 

df.drop(categorical_variables,axis=1,inplace=True)

payment_variables = []
spending_variables = []
delinquency_variables = []
balance_variables = []
risk_variables = []
for column in df.columns:
    if 'P' in column:
        payment_variables.append(column)
    if 'S' in column:
        spending_variables.append(column)
    if 'D' in column and 'ID' not in column:
        delinquency_variables.append(column)
    if 'B' in column:
        balance_variables.append(column)
    if 'R' in column:
        risk_variables.append(column)

In [4]:
df.drop('customer_ID',axis=1,inplace=True)

In [5]:
df = df.drop('S_2',axis=1)\
.drop('D_42',axis=1) \
.drop('D_49',axis=1) \
.drop('D_53',axis=1) \
.drop('D_73',axis=1) \
.drop('D_76',axis=1) \
.drop('R_9',axis=1)  \
.drop('D_82',axis=1) \
.drop('B_29',axis=1) \
.drop('D_87',axis=1) \
.drop('D_88',axis=1) \
.drop('D_106',axis=1)\
.drop('R_26',axis=1) \
.drop('D_108',axis=1)\
.drop('D_110',axis=1)\
.drop('D_111',axis=1)\
.drop('B_39',axis=1) \
.drop('B_42',axis=1) \
.drop('D_132',axis=1)\
.drop('D_134',axis=1)\
.drop('D_135',axis=1)\
.drop('D_136',axis=1)\
.drop('D_137',axis=1)\
.drop('D_138',axis=1)\
.drop('D_142',axis=1)\


delinquency_variables_redux = delinquency_variables.copy()



delinquency_variables_redux.remove('D_42') 
delinquency_variables_redux.remove('D_49') 
delinquency_variables_redux.remove('D_53') 
delinquency_variables_redux.remove('D_73') 
delinquency_variables_redux.remove('D_76') 
delinquency_variables_redux.remove('D_82') 
delinquency_variables_redux.remove('D_87') 
delinquency_variables_redux.remove('D_88') 
delinquency_variables_redux.remove('D_106')
delinquency_variables_redux.remove('D_108')
delinquency_variables_redux.remove('D_110')
delinquency_variables_redux.remove('D_111')
delinquency_variables_redux.remove('D_132')
delinquency_variables_redux.remove('D_134')
delinquency_variables_redux.remove('D_135')
delinquency_variables_redux.remove('D_136')
delinquency_variables_redux.remove('D_137')
delinquency_variables_redux.remove('D_138')
delinquency_variables_redux.remove('D_142')


df.loc[:,delinquency_variables_redux].fillna(0,inplace=True)

for col in df.columns:
    print(col)
    df[col].fillna(df[col].mean(),inplace=True)

Unnamed: 0
P_2
D_39
B_1
B_2
R_1
S_3
D_41
B_3
D_43
D_44
B_4
D_45
B_5
R_2
D_46
D_47
D_48
B_6
B_7
B_8
D_50
D_51
B_9
R_3
D_52
P_3
B_10
S_5
B_11
S_6
D_54
R_4
S_7
B_12
S_8
D_55
D_56
B_13
R_5
D_58
S_9
B_14
D_59
D_60
D_61
B_15
S_11
D_62
D_65
B_16
B_17
B_18
B_19
B_20
S_12
R_6
S_13
B_21
D_69
B_22
D_70
D_71
D_72
S_15
B_23
P_4
D_74
D_75
B_24
R_7
D_77
B_25
B_26
D_78
D_79
R_8
S_16
D_80
R_10
R_11
B_27
D_81
S_17
R_12
B_28
R_13
D_83
R_14
R_15
D_84
R_16
S_18
D_86
R_17
R_18
B_31
S_19
R_19
B_32
S_20
R_20
R_21
B_33
D_89
R_22
R_23
D_91
D_92
D_93
D_94
R_24
R_25
D_96
S_22
S_23
S_24
S_25
S_26
D_102
D_103
D_104
D_105
D_107
B_36
B_37
R_27
D_109
D_112
B_40
S_27
D_113
D_115
D_118
D_119
D_121
D_122
D_123
D_124
D_125
D_127
D_128
D_129
B_41
D_130
D_131
D_133
R_28
D_139
D_140
D_141
D_143
D_144
D_145
target


#### Armado del train_test

In [6]:
train_variables = []

for v in payment_variables:
    if v in df.columns:
        train_variables.append(v)
for v in balance_variables:
    if v in df.columns:        
        train_variables.append(v)
for v in risk_variables:
    if v in df.columns:        
        train_variables.append(v)     
for v in spending_variables:
    if v in df.columns:        
        train_variables.append(v)        
        
        

X = df.iloc[0:20000,:][train_variables]
#X = df[train_variables]
        
#X = df.drop(['target'],axis=1)
#X = X.drop(delinquency_variables_redux,axis=1)
#X = df[payment_variables]
y = df.iloc[0:20000,:].target
#y = df.target
features = X.columns





X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X_train_norm=scaler.fit_transform(X_train)
X_test_norm=scaler.fit_transform(X_test)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 85 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   P_2     20000 non-null  float64
 1   P_3     20000 non-null  float64
 2   P_4     20000 non-null  float64
 3   B_1     20000 non-null  float64
 4   B_2     20000 non-null  float64
 5   B_3     20000 non-null  float64
 6   B_4     20000 non-null  float64
 7   B_5     20000 non-null  float64
 8   B_6     20000 non-null  float64
 9   B_7     20000 non-null  float64
 10  B_8     20000 non-null  float64
 11  B_9     20000 non-null  float64
 12  B_10    20000 non-null  float64
 13  B_11    20000 non-null  float64
 14  B_12    20000 non-null  float64
 15  B_13    20000 non-null  float64
 16  B_14    20000 non-null  float64
 17  B_15    20000 non-null  float64
 18  B_16    20000 non-null  float64
 19  B_17    20000 non-null  float64
 20  B_18    20000 non-null  float64
 21  B_19    20000 non-null  float64
 22

### Hiperparametros lineales y polinomicos para el svm

In [7]:
#Seteo hyperparametros lineales y polinomicas
param_dist_polinomic = {"kernel" : ['poly'] ,
            "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9],
            "degree": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4],
            "gamma": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4],
            "coef0": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4]
              }

#param_dist_linear = {"kernel":['linear'],
#             "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9] }

param_dist_linear = {
             "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9] }

In [8]:
random_search_polynomic_svc = RandomizedSearchCV(SVC(random_state=0),
                                       param_distributions = param_dist_polinomic,
                                       n_iter = 2,
                                       scoring = 'roc_auc',
                                       cv = 5,
                                       n_jobs = 2)



random_search_linear_svc = RandomizedSearchCV(SVC(random_state=0),
                                       param_distributions = param_dist_linear,
                                       n_iter = 2,
                                       scoring = 'roc_auc',
                                       cv = 5,
                                       n_jobs = 2)

In [9]:
random_search_linear_svc.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__break_ties', 'estimator__cache_size', 'estimator__class_weight', 'estimator__coef0', 'estimator__decision_function_shape', 'estimator__degree', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__probability', 'estimator__random_state', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_iter', 'n_jobs', 'param_distributions', 'pre_dispatch', 'random_state', 'refit', 'return_train_score', 'scoring', 'verbose'])

## Entrenamiento de los modelos (sin probar porque no me los termina)

In [10]:
#%time
#random_search_linear_svc.fit(X_train, y_train) #descomentar


#random_search_polynomic_svc.fit(X_train,y_train)

#### Conseguimos los nuevos hiperparametros

In [11]:
#params_opt_linear_svm = random_search_linear_svc.best_params_
#params_opt_linear_svm

#params_opt_polynomial_svm = random_search_polynomic_svc.best_params_

In [12]:
#linear_svc = SVC().set_params(**params_opt_linear_svm)
linear_svc = SVC(C=1,tol=1e-4,max_iter=10000)
#polynomial_svc = SVC().set_params(**params_opt_polynomial_svm)


## Entrenamos de vuelta con svm y los nuevos parametros

In [13]:
linear_svc.fit(X_train_norm,y_train)

#polynomial_svc.fit(X_train,y_train)

SVC(C=1, max_iter=10000, tol=0.0001)

#### Muestra de los resultados

In [14]:
#AGREGAR METODOS DE RECALL, PRECICSION Y FSCORE
y_pred = linear_svc.predict(X_train_norm)
y_pred_test = linear_svc.predict(X_test_norm)



#Usando el entrenado
print("precision = ", precision_score(y_train, y_pred))
print("recall = ", recall_score(y_train, y_pred))
print("auc = ", roc_auc_score(y_train, y_pred))
print("f1 score = ", f1_score(y_train, y_pred))


confusion_matrix(y_train, y_pred)

precision =  0.8122481869460113
recall =  0.74795943606233
auc =  0.8447499781298519
f1 score =  0.778779294360031


array([[11258,   699],
       [ 1019,  3024]])

In [16]:
#Usando el test

confusion_matrix(y_test, y_pred_test)


print("precision = ", precision_score(y_test, y_pred_test))
print("recall = ", recall_score(y_test, y_pred_test))
print("auc = ", roc_auc_score(y_test, y_pred_test))
print("f1 score = ", f1_score(y_test, y_pred_test))

precision =  0.7080867850098619
recall =  0.7144278606965174
auc =  0.8077982375268897
f1 score =  0.7112431896978703
